### Настройка компонентов конвейера под свои нужды

Что если конвеер распознает что-то не так как нам нужно. Допустим как страну, а не название компаниии.

In [84]:
import spacy as sp
nlp = sp.load('ru_core_news_sm')

In [85]:
doc = nlp(u'Я хочу купить Москву')
for ent in doc.ents:
    print(ent.text, ent.label_)

Москву LOC


А у нас допустим Москва это название конфет

Компонент для распознавания сущностей реализован в API библиотеки spaCy в виде класса EntityRecognizer.С помощью методов этого
класса можно инициализировать экземпляр ner и применить его
к тексту. В большинстве случаев описывать операции явным образом
не нужно: spaCy делает это автоматически при создании объекта nlp,
а затем и объекта Doc соответственно.

In [86]:
# но придётся явным образом работать собъектом ner.
# Сначала добавим новую метку  - "название конфет" SWEET
# Затем подадим обучающийся набор данных, чтобы модель поняла, что такое "название конфет" SWEET

LABEL = 'SWEET'

# тренировочные данные. 
# (17, 23.. - место где стоит та самая метка которую мы вводим
TRAIN_DATA = [
    ('Нам нужно купить Москву', {'entities': [(17, 23, 'SWEET')]}), 
    ('Я хочу пить', {'entities': []}) # здесь нет элемента этого. Это важно для обучения
]


In [87]:
# получаем экземпляр конвеера ner
ner = nlp.get_pipe('ner')

# добавим метку 
ner.add_label(LABEL)

1

Прежде чем приступить к обучению средства распознавания сущностей, необходимо отключить остальные конвейеры, чтобы во время
обучения обновлялся только компонент распознавания сущностей:

In [88]:
meta_on = nlp.meta['pipeline'] #список компонентов конвейера, используемых с моделью
print(meta_on)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [89]:
# отключаем компоненты 
for component in meta_on:
    if component != 'ner':
        nlp.disable_pipes(component)

Примеры данных во время обучения демонстрируютсямодели в цикле
в случайномпорядке, чтобы какможно эффективнее обновлять данные
модели и избегать каких-либо обобщающих выводов из очередности
обучающих примеров. Выполнение этого кода займет некоторое время

In [90]:
# теперь можно начать обучение

from spacy.training.example import Example

for i in range(25):
    random.shuffle(TRAIN_DATA)
    for batch in sp.util.minibatch(TRAIN_DATA, size=2):
        for text, annotations in batch:
            # create Example
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update([example])

In [94]:

doc = nlp(u'Хочу скушать Москву')
for ent in doc.ents:
    print(ent.text, ent.label_)

doc = nlp(u'Ехать в Москва')
for ent in doc.ents:
    print(ent.text, ent.label_)

# теперь москва это не локация а конфеты



Москву SWEET
Москва SWEET


Учтите: внесенные только что обновления будут утрачены сразу после
закрытия текущего сеанса интерпретатора Python. Для решения этой
проблемы в классе Pipe, родительском для класса EntityRecognizer
и других классов компонентов конвейера, предусмотрен метод to_
disk(), предназначенный для сериализации конвейера на диск:

In [ ]:
#ner.to_disk('/usr/to/ner')
'''

загрузка
import spacy
from spacy.pipeline import EntityRecognizer
nlp = spacy.load('en', disable=['ner'])

ner = EntityRecognizer(nlp.vocab)
ner.from_disk('/usr/to/ner')
nlp.add_pipe(ner)
'''

Вы загрузили модель, отключив ее компонент ner по умолчанию .
Затем создали новый экземпляр ner , после чего загрузили данные
с диска . Добавили компонент ner в конвейер обработки .